In [2]:
import warnings
warnings.filterwarnings('ignore')
import pickle
import argparse
import time
import sys
import numpy as np
import pandas as pd
import os
import copy
import matplotlib.pyplot as plt
rootDirectory = "/homes/gws/kdeeds/FrequencyBounds"

In [4]:
sys.path.append(rootDirectory + "/BayesCard")
from Schemas.imdb.schema import gen_job_light_imdb_schema
schema = gen_job_light_imdb_schema(rootDirectory +"/Data/JOB")
ensemble_location = rootDirectory + "/StatObjects/BayesCardEnsembles/JOBLight/"
from DataPrepare.query_prepare_BayesCard import prepare_join_queries
query_filename = rootDirectory + "/Workloads/JOBLightQueriesBayes.sql" 
parsed_queries, true = prepare_join_queries(schema, ensemble_location, pairwise_rdc_path=None, 
                                            query_filename=query_filename, true_card_exist=True)

In [5]:
from Models.BN_ensemble_model import BN_ensemble
def load_ensemble(schema, model_path=ensemble_location):
    bn_ensemble = BN_ensemble(schema)
    for file in os.listdir(model_path):
        if file.endswith(".pkl"):
            print(file)
            with open(model_path+file, "rb") as f:
                bn = pickle.load(f)
                bn.infer_algo = "exact-jit"
                bn.init_inference_method()
            bn_ensemble.bns.append(bn)
            #bn_ensemble.bns[int(file[0])] = bn
    return bn_ensemble
bn_ensemble = load_ensemble(schema)
with open(query_filename, "rb") as f:
    real_query = f.readlines()
queries = bn_ensemble.parse_query_all(parsed_queries)

4_chow-liu_1.pkl
3_chow-liu_1.pkl
1_chow-liu_1.pkl
0_chow-liu_1.pkl
2_chow-liu_1.pkl


In [8]:
latency = []
q_errors = []
preds = []
for i, q in enumerate(queries):
    print(f"predicting query no {i}: {real_query[i]} \n")
    tic = time.time()
#    try:
    pred = bn_ensemble.cardinality(q)
    #except:
    #    print(f"query {i} is invalid!!!!!!!!!!!!!!!!!!!!!!!!!!!")
        # this query itself is invalid or it is not recognizable by the learnt BN
    #    continue
    latency.append(time.time() - tic)
    if pred is None or pred <= 1:
        pred = 1
    error = max(pred / true[i], true[i] / pred)
    print(f"true cardinality {true[i]}, predicted {pred} with q-error {error}")
    q_errors.append(error)
    preds.append(pred)
print("=====================================================================================")
for i in [50, 90, 95, 99, 100]:
    print(f"q-error {i}% percentile is {np.percentile(q_errors, i)}")
print(f"average latency is {np.mean(latency)*1000} ms")

predicting query no 0: b'SELECT COUNT(*) FROM movie_companies mc,title t,movie_info_idx mi_idx WHERE t.id=mc.movie_id AND t.id=mi_idx.movie_id AND mi_idx.info_type_id=112 AND mc.company_type_id=2||715\n' 

true cardinality 715, predicted 3536.0838050461816 with q-error 4.945571755309345
predicting query no 1: b'SELECT COUNT(*) FROM movie_companies mc,title t,movie_info_idx mi_idx WHERE t.id=mc.movie_id AND t.id=mi_idx.movie_id AND mi_idx.info_type_id=113 AND mc.company_type_id=2 AND t.production_year>2005 AND t.production_year<2010||9\n' 

true cardinality 9, predicted 19.90215808474934 with q-error 2.211350898305482
predicting query no 2: b'SELECT COUNT(*) FROM movie_companies mc,title t,movie_info_idx mi_idx WHERE t.id=mc.movie_id AND t.id=mi_idx.movie_id AND mi_idx.info_type_id=112 AND mc.company_type_id=2 AND t.production_year>2010||47\n' 

true cardinality 47, predicted 161.09078182570047 with q-error 3.42746344310001
predicting query no 3: b'SELECT COUNT(*) FROM movie_companies m

In [17]:
import re
re.sub("('....-..-.. ..:..:..')", r'\1::timestamp', "'2020-01-01 01:01:01'") 

"'2020-01-01 01:01:01'::timestamp"

In [ ]:
sys.path.append('/home/ziniu.wzn/FSPN/fspn')
from Data_prepare.schemas import gen_stats_light_schema
from Data_prepare.query_prepare_BayesCard import prepare_join_queries
schema = gen_stats_light_schema("/home/ziniu.wzn/stats")
hdf_path = "/home/ziniu.wzn/stats/gen_single_light"
meta_data_path = hdf_path + '/meta_data.pkl'

ensemble_location = "/home/ziniu.wzn/stats/FSPN_ensemble/"
#ensemble_location = "/home/yuxing.hyx/dataset/imdb-benchmark/spn_ensembles/ensemble_join_3_budget_5_10000000.pkl"
query_filename = "/home/ziniu.wzn/deepdb-stats/benchmarks/stats/stats_sub_queries.sql"
#query_filename = "/home/ziniu.wzn/deepdb-public/benchmarks/job-light/sql/job_light_queries.sql"
pairwise_rdc_path = "/home/yuxing.hyx/repository/imdb-benchmark/spn_ensembles/pairwise_rdc.pkl"
parsed_queries, true = prepare_join_queries(schema, ensemble_location, pairwise_rdc_path, query_filename, 
                                      join_3_rdc_based=True, true_card_exist=True)

In [ ]:
sys.path.append('/home/ziniu.wzn/BayesCard')
from Models.BN_ensemble_model import BN_ensemble
def load_ensemble(schema, model_path="/home/ziniu.wzn/stats/BayesCard_ensemble/"):
    bn_ensemble = BN_ensemble(schema)
    for file in os.listdir(model_path):
        if file.endswith(".pkl"):
            print(file)
            with open(model_path+file, "rb") as f:
                bn = pickle.load(f)
                bn.infer_algo = "exact-jit"
                bn.init_inference_method()
            bn_ensemble.bns[int(file[0])] = bn
    return bn_ensemble
bn_ensemble = load_ensemble(schema)

In [ ]:
with open(query_filename, "rb") as f:
    real_query = f.readlines()

queries = bn_ensemble.parse_query_all(parsed_queries)


In [ ]:
latency = []
q_errors = []
for i, q in enumerate(queries):
    tic = time.time()
    try:
        pred = bn_ensemble.cardinality(q)
    except:
        print(f"query {i} is invalid!!!!!!!!!!!!!!!!!!!!!!!!!!!")
        # this query itself is invalid or it is not recognizable by the learnt BN
        continue
    latency.append(time.time() - tic)
    if pred is None or pred <= 1:
        pred = 1
    error = max(pred / true[i], true[i] / pred)
    print(f"predicting query no {i}: {real_query[i]} \n")
    print(f"true cardinality {true[i]}, predicted {pred} with q-error {error}")
    q_errors.append(error)
print("=====================================================================================")
for i in [50, 90, 95, 99, 100]:
    print(f"q-error {i}% percentile is {np.percentile(q_errors, i)}")
print(f"average latency is {np.mean(latency)*1000} ms")

In [ ]:
parsed_queries[1869]

In [ ]:
q_errors = np.asarray(q_errors)

In [ ]:
np.where(q_errors > 5000)

In [ ]:
q = queries[1869]

In [ ]:
print(len(q))
q

In [ ]:
real_query[1869]

In [ ]:
bn_ensemble.bns[1].table_name

In [ ]:
bn_ensemble.bns[0].

In [ ]:
parsed_queries[1869]

In [ ]:
p, _ = bn_ensemble.bns[1].query(q[1]["query"], n_distinct=q[1]["n_distinct"], return_prob=True)

In [ ]:
p

In [ ]:
p, _ = bn_ensemble.bns[1].expectation(q[1]["query"], e, n_distinct=q[1]["n_distinct"], return_prob=True)

In [ ]:
q[0]*p

In [ ]:
e = ['posts.mul_postHistory.PostId', 'posts.mul_comments.PostId_nn']

In [ ]:
bn_ensemble.bns[1].fanout_attr_inverse

In [ ]:
pred = np.load("/home/ziniu.wzn/stats/est_results/bayesCard_join.npy")

In [ ]:
query_filename = "/home/ziniu.wzn/deepdb-stats/benchmarks/stats/stats_CEB_sub_queries.sql"
with open(query_filename, "r") as f:
    queries = f.readlines()
true = [int(q.split("||")[-1]) for q in queries]
true = np.asarray(true)

In [ ]:
q_errors = np.maximum(true/pred, pred/true)

In [ ]:
for i in [50, 90, 95, 99, 100]:
    print(f"q-error {i}% percentile is {np.percentile(q_errors, i)}")

In [ ]:
pred2 = pred + np.random.normal(10, 100, size=2603)
pred2[pred2<=1] = 1

In [ ]:
q_errors = np.maximum(true/pred2, pred2/true)
for i in [50, 90, 95, 99, 100]:
    print(f"q-error {i}% percentile is {np.percentile(q_errors, i)}")

In [ ]:
np.save("/home/ziniu.wzn/stats/est_results/bayesCard_join_update", pred2)